### Initial Signal Exploration

*Date:* 2025-06-19 <br> 
*Author:* salil apte  <br> 
*Version:* 1.0  <br> 
*Filename:* `01-initial-signal-exploration.ipynb`

This notebook contains the initial steps in exploring raw signal data. It includes basic filtering and visualization to guide subsequent analysis and modeling.

In [6]:
import pandas as pd
import numpy as np
from scipy.signal import firwin, filtfilt
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [7]:
# Load the training data
df = pd.read_csv(r"E:\repos\vital-sign-estimation\data\raw\train.csv")

In [8]:
# Extract the first five PPG signals to filter and visualize
ppg_signals = df.iloc[0:5,0:3000].values
fs = 100 # 100hz sampling
n_samples =ppg_signals.shape[1]
# Timestamps
t = np.linspace(0, n_samples / fs, n_samples, endpoint=False)

In [9]:
# FIR filter design
lowcut = 0.5  # Hz 30 bpm
highcut = 4.0  # Hz 240 bpm
numtaps = 201

low = lowcut / (fs / 2)
high = highcut / (fs / 2)
fir_coeff = firwin(numtaps, [low, high], pass_zero=False)

# Apply zero-phase filtering along axis=1 (each row)
ppg_signals_ac_filt = np.array([
    filtfilt(fir_coeff, 1.0, signal)
    for signal in ppg_signals
])

In [11]:
# Create 5 vertical subplots to visualize the PPG signals
fig = make_subplots(rows=5, cols=1, shared_xaxes=True, vertical_spacing=0.02,
                    subplot_titles=[f'Filtered PPG Signal {i+1}' for i in range(5)])

colors = ['red', 'blue', 'green', 'orange', 'magenta']

for i in range(5):
    fig.add_trace(
        go.Scatter(x=t, y=ppg_signals_ac_filt[i], mode='lines', line=dict(color=colors[i])),
        row=i+1, col=1
    )
    fig.update_yaxes(title_text="Amplitude", row=i+1, col=1)

fig.update_layout(
    height=1000, width=900,
    title_text="Bandpass FIR Filtered PPG Signals (0.5–4 Hz)",
    showlegend=False,
    template='plotly_white'
)
fig.update_xaxes(title_text="Time (seconds)", row=5, col=1)

fig.show()